In [ ]:
from huggingface_hub import login

login("")

In [ ]:
!pip install llava

In [ ]:
import transformers
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import os
import json
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
processor = AutoProcessor.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import sys

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a FA24-CV folder and put all the files under A2_PartA folder, then "FA24-CV/A2_PartA"
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'place_centric_images'

GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))

In [ ]:
images_path = os.path.join(GOOGLE_DRIVE_PATH)
image_files = [f for f in os.listdir(images_path) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

# print(image_files)

In [ ]:
# image_dir = "./images"

In [ ]:
import json
from collections import Counter
with open("./qa_pairs.json", "r") as f:
    qa_pairs = json.load(f)
first_key = next(iter(qa_pairs))
first_value = qa_pairs[first_key]
count = 0
attributes = set()
places = set()
attributes_counter = Counter()
places_counter = Counter()
for k in qa_pairs:
  for attr in qa_pairs[k]['attributes']:
      attributes.add(attr)
      # count = count + 1
      attributes_counter[attr] += 1
for k in qa_pairs:
  for attr in qa_pairs[k]['place']:
      places.add(attr)
      count = count + 1
      places_counter[attr] += 1

unique_count = len(attributes_counter)
unique_count1 = len(places_counter)

# print("Key:", first_key)
# print("Value:", first_value)
# print(unique_count1)
# print(count)


Key: ChIJvcEkGIpZwokRY8mfbXLPlu4
Value: {'question': 'Which human intentions can be accomplished here?', 'place': 'Bank of America Financial Center', 'attributes': ['bank'], 'answer': 'C', 'answer_a': 'Organize a movie screening event.', 'answer_b': 'Perform medical surgery.', 'answer_c': 'Deposit or withdraw money.', 'answer_d': 'Host a large wedding reception.'}
757
113428


In [ ]:
# # image_name = "/content/ChIJ__8vRBHJ5zsRn4vojEnCMrI.jpg"
# # question = ("What is the place? Please choose and answer with one of the following attributes: "
# #             + ", ".join(attributes) + ". Only provide one short answer.")


# gt_attr = "home goods store"
# results = {}

# # 加载本地图片
# try:
#     image = Image.open(image_name).convert("RGB")
# except Exception as e:
#     print(f"Error loading image {image_name}: {e}")
#     results[image_name] = {
#         "answer": "",
#         "ground_truth": gt_attr,
#         "correctness": "unknown"
#     }
# else:
#     # 构建对话
#     conversation = [
#         {
#         "role": "system",
#         "content": f"The following attributes are relevant for this task: {', '.join(attributes)}."
#     },
#         {
#             "role": "user",
#             "content": [
#                 {"type": "image"},
#                 {"type": "text", "text":"What is the place? Please choose one relevant attribute as your answer."}
#             ],
#         }
#     ]
#     # 转化对话为模型可用的prompt
#     prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

#     # 将图像与文本一并处理
#     inputs = processor(images=[image], text=[prompt], padding=True, return_tensors="pt").to(device, torch.float16)

#     # 模型推理
#     with torch.no_grad():
#         output_ids = model.generate(**inputs, max_new_tokens=5, do_sample=True, temperature=0.7)
#     answer = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

#     # 简单匹配判断
#     if gt_attr.lower() in answer.lower():
#         correctness = "correct"
#     else:
#         correctness = "wrong"

#     results[image_name] = {
#         "answer": answer,
#         "ground_truth": gt_attr,
#         "correctness": correctness
#     }

# # 保存结果
# with open("final_results.json", "w") as f:
#     json.dump(results, f, indent=4)

# print("Done! Results saved to final_results.json")

Done! Results saved to final_results.json


In [ ]:

results1 = {}
c = 0
d = 0

for img_name in tqdm(image_files, desc="Processing images"):

    img_path1 = os.path.join(images_path, img_name)
    img_name = os.path.splitext(img_name)[0]
    gt_attr = qa_pairs[img_name]["attributes"]
    gt_answer = qa_pairs[img_name]["answer"]
    qa_info = qa_pairs.get(img_name, {})

    options = {key[-1].upper(): value for key, value in qa_info.items() if key.startswith("answer_")}
    options_text = "\n".join([f"{key}: {value}" for key, value in options.items()])

    try:
        image = Image.open(img_path1).convert("RGB")
    except Exception as e:
        print(f"Error loading image {img_name}: {e}")
        results[img_name] = {
            "answer": "",
            "ground_truth": gt_attr,
            "correctness": "unknown"
        }
        continue
    conversation = [
        {
        "role": "system",
        "content": """
           You are the Visual Instructor, an expert in visual reasoning and critical thinking. Your role is to guide your students (less advanced AI models) in accurately interpreting visual information and answering related questions.
           Analyze the provided image and question, and develop a clear, step-by-step reasoning chain. Your instructions must be logical, easy to follow, and comprehensive, enabling your students to interpret images and derive correct answers independently.
           Analyze the image and choose the correct option.
           Output Format:
          - Each step must be a single sentence.
          - Separate steps with a newline (`\n`).
          - First step is to Only respond with the letter of the option (A, B, C, or D).
          - Conclude the reasoning with an instruction to synthesize the steps into an answer.
    """},
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text":f"What is the place?,Here are the options:\n{options_text}\n\n."}
            ],
        }
    ]

    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


    inputs = processor(images=[image], text=[prompt], padding=True, return_tensors="pt").to(device, torch.float16)


    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.2)
    raw_answer = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()



    answer = raw_answer.strip().upper()
    if "ASSISTANT:" in answer:
      answer = raw_answer.split("ASSISTANT:")[1].strip()
      answer = answer.split()[0]
      answer = answer.rstrip(':')
    else:
      answer = raw_answer.strip()
    print(answer)

    if answer in ["A", "B", "C", "D"]:

        if answer == gt_answer:
            correctness = "correct"
            c += 1
        else:
            correctness = "wrong"
            d += 1
    else:
        correctness = "invalid"
        d += 1


    results1[img_name] = {
        "raw_answer": raw_answer,
        "answer": answer,
        "ground_truth": gt_answer,
        "options": options,
        "correctness": correctness
    }



# 保存结果
with open("final_results1.json", "w") as f:
    json.dump(results1, f, indent=4)

print("Done! Results saved to final_results.json")
print(c)
print(d)

Processing images:   0%|          | 1/5828 [00:00<34:23,  2.82it/s]

B


Processing images:   0%|          | 2/5828 [00:00<34:34,  2.81it/s]

B


Processing images:   0%|          | 3/5828 [00:01<34:52,  2.78it/s]

...

B


Processing images: 100%|█████████▉| 5827/5828 [1:53:12<00:01,  1.34s/it]

B


Processing images: 100%|██████████| 5828/5828 [1:53:13<00:00,  1.17s/it]

B
Done! Results saved to final_results.json
5089
739
